In [1]:
import tensorflow as ts
import pandas as pd
import numpy as np
import seaborn as sn
import os as os

In [17]:
df.head()

,Unnamed: 0,Team,Conference,Region,Cinderella,Conference Tournament Champion,Number of Tournament Wins,Made Tournament Previous Year,Game Count,Wins,...,Assist to Turnover Ratio,ESPN Strength of Schedule,Quad 1 Wins,Quad 1 Losses,Total Points,Average PPG,Total Opp Points,Average Opp PPG,Total Scoring Differential,Scoring Differential Per Game
0,0,Baylor,B12,South,0.0,0.0,6.0,1.0,23.0,22.0,...,1.372483,68.0,8.0,2.0,2025.0,84.375000,1594.0,66.416664,431.0,17.958334
1,1,Colgate,Pat,South,1.0,1.0,0.0,1.0,14.0,14.0,...,1.735099,332.0,0.0,0.0,1295.0,86.333336,1029.0,68.599998,266.0,17.733334
2,2,Liberty,ASun,Midwest,1.0,1.0,0.0,1.0,27.0,23.0,...,1.645756,298.0,0.0,0.0,2097.0,74.892860,1670.0,59.642857,427.0,15.250000
3,3,Florida St.,ACC,East,0.0,0.0,2.0,1.0,21.0,16.0,...,1.022222,48.0,3.0,4.0,1736.0,78.909088,1547.0,70.318184,189.0,8.590909
4,4,Oral Roberts,Sum,South,1.0,1.0,2.0,0.0,25.0,16.0,...,1.100334,165.0,0.0,0.0,2126.0,81.769234,1970.0,75.769234,156.0,6.000000


## Clean Data

## Create Model

## Validate Results